# Dependencies 

In [ ]:
## Setup
# import environment
import gym_super_mario_bros
# import joypad
from nes_py.wrappers import JoypadSpace
# import controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

## Preprocessing
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv
from matplotlib import pyplot as plt

## Agent training
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
# from stable_baselines.common import set_global_seeds
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
# from stable_baselines.common.policies import MlpPolicy
# callback
import numpy as np
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor

In [ ]:
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0", render_mode='human', apply_api_compatibility=True)
env.action_space

In [ ]:
# wrap environment with controls
env = JoypadSpace(env, [["right"], ["right", "A"]])
env.action_space


In [ ]:
# Grayscale
print("Input shape before grayscale: ", env.observation_space.shape)
class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def permute_orientation(self, observation):
        # permute [H, W, C] array to [C, H, W] tensor
        observation = np.transpose(observation, (2, 0, 1))
        observation = torch.tensor(observation.copy(), dtype=torch.float)
        return observation

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        transform = T.Grayscale()
        observation = transform(observation)
        return observation
print("Input shape after grayscale: ", env.observation_space.shape)


In [ ]:
env = SkipFrame(env, skip=4)
env.reset()
env = GrayScaleObservation(env)
env.reset()
env = ResizeObservation(env, shape=84)
env.reset()
env = FrameStack(env, num_stack=4)
env.reset()

In [ ]:
CHECKPOINT_PATH = "./checkpoints_pytorch"

# Testing the agent

#

In [ ]:
# load the agent
model = PPO.load(CHECKPOINT_PATH + "/model_1000000")

In [ ]:
state = env.reset()
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()